In [58]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
from scipy.spatial import distance
import subprocess

folder = r'D:\datos_GPetersii\datos_GPetersii\Fish3\Object\raw\50fps'
os.chdir(folder)
files = sorted(glob.glob('*clean.h5'))
videos = sorted(glob.glob('*.avi'))
print('hay ' + str(len(files)) + ' archivos')

hay 48 archivos


In [59]:
files = files[:12]
videos = videos[:12]
print(files, videos)

['video2023-07-01T16_50_07_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T17_10_07_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T17_30_07_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T17_50_07_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T18_10_08_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T18_30_08_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T18_50_08_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T19_10_08_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T19_30_08_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T19_50_08_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T20_10_08_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5', 'video2023-07-01T20_30_09_50fpsDLC_resnet50_test2Jun21shuffle1_200000_clean.h5'] ['video2023-07-01T16_50_07_

In [60]:
def get_centroids(track, n_keypoints):

    centroids = []

    bodyparts = np.array([x for x in track.columns.get_level_values(1)])
    xpositions = pd.DataFrame(np.zeros((len(track),n_keypoints)), columns=np.unique(bodyparts))
    for i in range(n_keypoints):
        xpositions[np.unique(bodyparts)[i]] = (track[track.columns.get_level_values(0)[0], np.unique(bodyparts)[i], 'x'])
        
    median_xposition = np.median(xpositions, axis=1) #xpos

    ypositions = pd.DataFrame(np.zeros((len(track),n_keypoints)), columns=np.unique(bodyparts))
    for i in range(n_keypoints):
        ypositions[np.unique(bodyparts)[i]] = (track[track.columns.get_level_values(0)[0], np.unique(bodyparts)[i], 'y'])
        
    median_yposition = np.median(ypositions, axis=1) #ypos

    for j in range(len(median_xposition)):
        centroids.append([median_xposition[j], median_yposition[j]])

    return centroids

def calculate_velocity(x, sf, pix_to_cm):
    desplazamiento = [abs(distance.euclidean(x,y)/pix_to_cm) for x, y in zip(x[1:], x[:-1])]
    dt = 1 / sf
    v = [i/dt for i in desplazamiento]
    return v
def save_moment(file, start, end, track, i):
    output_video = file[:24] + '_moment_' + str(i) + '.h5'
    k = 'moment_' +str(i)
    moment_df = track.iloc[start:end+1, :]
    moment_df.to_hdf(output_video, key=k)

def extract_frames(input_video, output_video, start, end, fps):
    # Construct ffmpeg command
    if os.path.exists(output_video):
        print(f"{output_video} already exists. Skipping extraction.")
        return
    
    ffmpeg_cmd = [
        r'C:\ffmpeg\bin\ffmpeg.exe', 
        '-i', input_video,
        '-vf', f'select=\'between(n\,{start}\,{end})\',setpts=PTS-STARTPTS',
        '-r', str(fps),
        output_video
    ]
    
    # Execute ffmpeg command
    subprocess.run(ffmpeg_cmd)


In [61]:
for file, vid in zip(files, videos):
    os.chdir(folder)
    track = pd.read_hdf(file)
    centroids = get_centroids(track, n_keypoints=6)
    velocity = calculate_velocity(centroids, sf=150, pix_to_cm=12)
    vel_per_frame = pd.DataFrame(zip([round(x[0]/12) for x in centroids], [round(y[1]/12) for y in centroids], velocity), columns=['x', 'y', 'v'])
    idx_movement = [i for i,x in enumerate(velocity) if x > 10]
    duration_moments = [i-j for i,j in zip(idx_movement[1:], idx_movement[:-1])]
    single_moments = [1 if i < 50 else 0 for i in duration_moments] 

    start = [0]
    end = []
    for i, x in enumerate(single_moments[:-1]):
        if (x == 1 and single_moments[i-1] == 0):
            start.append(i)
        if (x == 1 and single_moments[i+1] == 0):
            end.append(i+1)
    frames_moments = [(idx_movement[x],idx_movement[y]) for x,y in zip(start, end) if not x==y]
    dif = [x[1]-x[0] for x in frames_moments]
    duration = round(np.median(dif))
    if duration > 100:
        os.chdir(r'C:\Users\Compras\Desktop\B-SOiD\Fish3')
        for i, moment in enumerate(frames_moments):
            medio = moment[1] - moment[0]
            start = medio - round(duration/2)
            end = medio + round(duration/2)
            if start > 0 and end < len(track):
                save_moment(file, start=start, end=end, track=track, i=i)
                output_vid =  vid[:24] + '_moment_' + str(i) + '.mp4'
                extract_frames(folder + "\\" + vid, output_vid, start=start, end=end, fps=50)
            

In [53]:
moment = pd.read_hdf(r"C:\Users\Compras\Desktop\B-SOiD\Fish2\video2023-06-22T19_00_09_moment_26.h5")
moment

scorer    DLC_resnet50_test2Jun21shuffle1_200000                         \
bodyparts                                   chin                          
coords                                         x           y likelihood   
1266                                  374.381256  239.509521   0.999818   
1267                                  374.351935  239.803812   0.999828   
1268                                  374.322614  240.098102   0.999839   
1269                                  374.293292  240.392392   0.999849   
1270                                  374.263971  240.686682   0.999860   
...                                          ...         ...        ...   
1542                                  367.225647  294.098267   0.999823   
1543                                  368.054169  292.256592   0.999843   
1544                                  364.432556  289.578857   0.999830   
1545                                  364.980560  291.438904   0.999896   
1546                                  365.629883  292.286926   0.999898   

scorer                                                                \
bodyparts       mouth                               head               
coords              x           y likelihood           x           y   
1266       360.825104  229.160370   0.999626  344.328796  217.822983   
1267       360.894537  229.804416   0.999598  344.565265  218.542978   
1268       360.963971  230.448462   0.999569  344.801733  219.262973   
1269       361.033405  231.092508   0.999541  345.038202  219.982968   
1270       361.102838  231.736554   0.999513  345.274670  220.702963   
...               ...         ...        ...         ...         ...   
1542       360.414612  280.149811   0.999680  350.665070  264.984497   
1543       359.946350  279.536865   0.999737  350.491974  264.771393   
1544       358.787598  279.101868   0.999766  350.001923  265.063904   
1545       359.109741  279.925995   0.999676  350.113312  265.889160   
1546       358.848358  280.247223   0.999618  349.832916  265.862915   

scorer                                                               \
bodyparts                  body1                              body2   
coords    likelihood           x           y likelihood           x   
1266        0.999534  319.895477  207.121750   0.999143  230.083099   
1267        0.999530  320.451721  207.685223   0.999233  230.581522   
1268        0.999526  321.007965  208.248697   0.999323  231.079944   
1269        0.999521  321.564209  208.812170   0.999413  231.578366   
1270        0.999517  322.120453  209.375644   0.999503  232.076788   
...              ...         ...         ...        ...         ...   
1542        0.999653  336.718719  246.337082   0.999911  277.542847   
1543        0.999592  335.534271  246.796570   0.999916  277.773102   
1544        0.999400  335.743134  246.447418   0.999879  276.980530   
1545        0.999426  335.903992  246.714508   0.999860  276.669647   
1546        0.999524  336.062347  246.742828   0.999870  276.728149   

scorer                                                               
bodyparts                               tail                         
coords              y likelihood           x           y likelihood  
1266       179.969254   0.991168  180.650970  170.593323   0.999246  
1267       179.802420   0.991723  181.019470  170.732220   0.999095  
1268       179.635587   0.992278  181.387970  170.871118   0.998945  
1269       179.468753   0.992833  181.756470  171.010016   0.998795  
1270       179.301920   0.993389  182.124969  171.148914   0.998644  
...               ...        ...         ...         ...        ...  
1542       173.172958   0.999327  246.506241  128.868973   0.998542  
1543       173.062088   0.999049  245.942657  131.575333   0.999507  
1544       171.223190   0.998567  244.184921  129.617111   0.998695  
1545       170.093338   0.998323  243.944168  130.140701   0.998950  
1546       169.788162   0.998135  2

La siguiente celda plotea la trayectoria para un momento dado. Con el tamano del punto proporcional a la velocidad del pez en ese punto.

In [75]:
import seaborn as sns
def plot_trayectories(vel_per_frame, start_frame, end_frame):
    data = vel_per_frame.iloc[start_frame:end_frame+1, :]
    #graficamos
    fig, ax = plt.subplots()
    points = sns.scatterplot(
        data=data,
        x='x',
        y='y',
        size='v',
        hue='v'
    )
    ax.plot(data['x'], data['y'], c='pink', linewidth=.5, linestyle='--', alpha=.5)
    ax.scatter(x =  439.2/12, y = 207.53/12, c='crimson', s=150, marker='*')
    ax.set_ylim([0,63])
    ax.set_xlim([0,63])
    plt.show()

In [135]:
from random import sample
rand_frames_moments = sample(frames_moments, 5)
rand_frames_moments

[(18561, 19609), (29131, 32429), (1, 17815), (55049, 55130), (36707, 40577)]

In [ ]:
for moment in rand_frames_moments:
    plot_trayectories(vel_per_frame, start_frame=moment[0] - 500, end_frame=moment[1] + 500)

Las siguientes celdas generan videos de muestra de las frames seleccionadas. 

In [143]:
import subprocess

def extract_frames(input_video, output_video, start_frame, end_frame, fps):
    # Construct ffmpeg command

    ffmpeg_cmd = [
        r'C:\ffmpeg', 
        '-i', input_video,
        '-vf', f'select=\'between(n\,{start_frame}\,{end_frame})\',setpts=PTS-STARTPTS',
        '-r', str(fps),
        output_video
    ]
    
    # Execute ffmpeg command
    subprocess.run(ffmpeg_cmd)

In [77]:
vid_folder = 'D:\datos_GPetersii\datos_GPetersii\Fish7\Object\raw'
videos = sorted(glob.glob(vid_folder+'*.avi'))

In [136]:
#vid_folder = '/Volumes/Expansion/datos_GPetersii/datos_GPetersii/Fish1/Object/raw/'
#videos = glob.glob(vid_folder+'*.avi')
#for vid in videos :
input_video = videos[0]
for i, moment in enumerate(rand_frames_moments):
    output_video = input_video[74:-9] + '_moment_' + str(i) + '.avi'
    start_frame = moment[0] - 100 # Start frame index
    end_frame = moment[1] + 100   # End frame index
    fps = 50           # Desired frames per second for output video

    extract_frames(input_video, output_video, start_frame, end_frame, fps)


FileNotFoundError: [WinError 2] El sistema no puede encontrar el archivo especificado